# Prompt Tuning GPT-J with 8-bit weights for Headline Generation

This notebook contains the code for prompt tuning a GPT-J Model using OpenPrompt on the task of Headline Generation from News Articles.

*   GPT-J-6B: A 6 billion parameter, autoregressive text generation model by EleutherAI trained on [The Pile](https://pile.eleuther.ai/) using [Mesh Transformer JAX](https://github.com/kingoflolz/mesh-transformer-jax/) (Ben Wang and Aran Komatsuzaki).
*   GPT-J-6B-8bit: A quantized GPT-J-6B with 8-bit weights for scalable and cost-efficient fine-tuning by Hivemind with [LoRA](https://arxiv.org/pdf/2106.09685.pdf) and [8-bit Adam](https://arxiv.org/abs/2110.02861).
*   Prompt Tuning: Latest paradigm to adapt pre-trained language models (PLMs) to downstream NLP tasks by learning and adding "soft prompts" to the model input (Brian Lester, Rami Al-Rfou, Noah Constant), [Paper](https://arxiv.org/abs/2104.08691).
*   OpenPrompt: An Open-Source Framework for Prompt-learning, [Code](https://arxiv.org/abs/2104.08691) [Docs](https://thunlp.github.io/OpenPrompt/) [Paper](https://arxiv.org/abs/2111.01998)

*Note: Running this notebook excedes the ressources of free google colab. \
For reproduction, we trained with Colab Pro Extended RAM and Premium GPU Settings. Extended RAM is necessary. Premium GPU is not, but it speeds up the training process bx factor x3.*

##Installs & Imports

Freeze library versions to make it reproducible.

In [1]:
!pip install -I transformers==4.21.0 
# Transformers is pinned to this specific version because it fixes bug 
# https://github.com/huggingface/transformers/issues/19290
!pip install openprompt==1.0.1
!pip install accelerate==0.15.0
!pip install bitsandbytes==0.35.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 14.5 MB/s 
     |████████████████████████████████| 772 kB 73.5 MB/s 
     |████████████████████████████████| 182 kB 94.6 MB/s 
     |████████████████████████████████| 78 kB 8.9 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 701 kB 86.8 MB/s 
     |████████████████████████████████| 6.6 MB 76.3 MB/s 
     |████████████████████████████████| 17.3 MB 85.9 MB/s 
     |████████████████████████████████| 140 kB 86.6 MB/s 
     |████████████████████████████████| 155 kB 89.6 MB/s 
     |████████████████████████████████| 61 kB 119 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.3 requires numpy<1.23.0,>=1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 146 kB 14.9 MB/s 
     |████████████████████████████████| 1.2 MB 93.1 MB/s 
     |████████████████████████████████| 452 kB 88.9 MB/s 
     |████████████████████████████████| 125 kB 102.0 MB/s 
     |████████████████████████████████| 212 kB 98.9 MB/s 
     |████████████████████████████████| 132 kB 95.7 MB/s 
     |████████████████████████████████| 16.9 MB 89.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.0
    Uninstalling numpy-1.24.0:
      Successfully uninstalled numpy-1.24.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 3.3.6 requires packaging<22.0.0dev,>=14.3, but you have packaging 22.0 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 191 kB 14.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62.5 MB 143.4 MB/s 


In [2]:
from openprompt.pipeline_base import PromptForGeneration
from openprompt.prompts.generation_verbalizer import GenerationVerbalizer
from tqdm import tqdm
from openprompt.data_utils import PROCESSORS
import torch
from openprompt.data_utils.utils import InputExample
import argparse
import numpy as np

from openprompt import PromptDataLoader
from openprompt.prompts import ManualVerbalizer
from openprompt.prompts import SoftTemplate
from openprompt import PromptForClassification
import time
import os
import re
from openprompt.utils.crossfit_metrics import evaluate as crossfit_evaluate

## Patches for Model conversion to 8 bits

We use an 8-bit version of EleutherAI's GPT-J-6B model converted with facebook's bitsandbytes library. This reduces the model's size from 24GB down to 6GB.

*    large weight tensors are quantized using dynamic 8-bit quantization and de-quantized just-in-time for multiplication 
*    using gradient checkpoints to store one only activation per layer: using dramatically less memory at the cost of 30% slower training

In [3]:
import transformers
import pandas as pd
import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd

import accelerate

from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise
from torch.utils.data import DataLoader
from bitsandbytes.optim import Adam8bit

In [4]:
class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias
 
    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
        if self.adapter:
            output += self.adapter(input)
        return output
 
    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"
 
 
class DequantizeAndLinear(torch.autograd.Function): 
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias).clone()
 
    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias
 
 
class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
 
    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantized weights and input indices are not differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output 
 
    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"
 
 
def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)
 
    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)
 
 
def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr(
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

In [5]:
class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)
        

class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock

## Prompt Tuning Setup

### Step 1: Define a task

We attempt to prompt tune for headline generation. This means we input a news articles fulltext and expect a headline like output. \
OpenPrompt provides the ```InputExample``` class for task/data definition. 
* ```text_a``` is the input text of the data, in our case a news articles fulltext
* ```tgt_text``` is the ground truth, in our case the news articles headline
* ```guid``` is a unique identifier for each input example


#### Option 1: Create small dataset manually (for testing purposes only)

In [6]:
# Create datasets manually
from openprompt.data_utils import InputExample
dataset = [ 
    # For simplicity, there's only four examples
    # text_a is the input text of the data
    # tgt_text is the headline target text
    # guid is a unique identifier for each input example.
    InputExample(
        guid = 1,
        text_a = "Noch vor wenigen Wochen war FTX eine der größten Kryptobörsen der Welt. Das Vermögen ihres Gründers Sam Bankman-Fried (SBF) wurde von Forbes und Bloomberg zeitweise mit über 26 Milliarden Dollar bewertet.\n\nFTX-Pleite: SBF als CEO zurückgetreten\n\nAnfang November 2022 ging dann alles ganz schnell: Innerhalb weniger Tage rutschte FTX in die Pleite – Milliarden Dollar wurden abgezogen, Rettungsversuche scheiterten, Bankman-Fried trat als Firmenchef zurück und meldete Insolvenz an.\n\nJetzt hat sich SBF zum ersten Mal seit Mitte November öffentlich zu Wort gemeldet. Von seinem Wohnsitz auf den Bahamas aus war er bei dem von der New York Times veranstalteten Dealbook Summit zugeschaltet.\n\nKomplett versagt, aber nicht betrogen?\n\nDabei zeigte sich der FTX-Pleitier reumütig: „Wir haben komplett versagt“, gestand Bankman-Fried ein. Zudem habe er nie versucht, jemanden zu betrügen. Er sei schockiert darüber, was aus dem florierenden Geschäft im vergangenen Monat geworden sei.\n\nWarum er, wie ihm vorgeworfen wird, Kundengelder in Milliardenhöhe von FTX abgezogen habe, um sein anderes Unternehmen, Alameda Research, zu retten, darauf ging Bankman-Fried nicht genauer ein.\n\nScharfe Kritik von SBF-Nachfolger bei FTX\n\nEin wenig Licht ins Dunkel bringt hier wohl eher der FTX-Chef John Ray, der im Zuge des Konkursverfahrens die SBF-Nachfolge antrat. Er habe noch nie in seiner Karriere „solch ein komplettes Versagen an Unternehmenskontrolle und so einen Mangel an vertrauenswürdigen Finanzinformationen erlebt“, wird Ray von Spiegel Online zitiert.\n\nWas in der Führungsriege bei FTX angestellt worden sei, sei „inakzeptabel“. Abzuwarten bleibt freilich, was bei den in den USA laufenden Ermittlungen und Sammelklagen gegen Bankman-Fried herauskommt.\n\nGenug Geld für US-Kunden von FTX da\n\nFür die FTX-Kund:innen in den USA hat Bankman-Fried derweil gute Nachrichten – wenn man dem trauen kann. Die US-Plattform sei solvent. FTX verfüge über genügend Geld, um die Kund:innen dort auszuzahlen.\n\nIhm selbst sei im Vergleich zu den finanziell rosigen Zeiten der vergangenen Monate nur noch wenig von seinem einstigen Vermögen geblieben. Auf einem Bankkonto habe er noch 100.000 Dollar, so SBF.",
        tgt_text = "Sam Bankman-Fried: Nach FTX-Pleite sind von 26 Milliarden nur noch 100.000 Dollar übrig"
    ),
    InputExample(
        guid = 2,
        text_a = "Am 8. Dezember 2022 haben Bundesverkehrsminister Volker Wissing und Deutsche-Bahn-Infrastrukturvorstand Berthold Huber eine Vereinbarung für eine Milliardenfinanzierung zur Modernisierung der Bahnstrecken unterzeichnet. Gefördert werden soll das Projekt „Digitale Schiene Deutschland“ mit etwa 2,7 Milliarden Euro durch das Bundesministerium für Digitales und Verkehr, kurz BMDV.\n\nEine Milliarde Euro mehr\n\nEs gab bereits einen bestehenden Finanzierungsvertrag in Höhe von etwa 1,7 Milliarden Euro – mit der jüngsten Vereinbarung wurde er um eine Milliarde erhöht. Zusätzlich gab es für die Deutsche Bahn (DB) noch Finanzierungszusagen für weitere Projekte.\n\nMit dem Projekt „Digitale Schiene Deutschland“ sollen Strecken modernisiert werden, ohne dass Gleise ausgetauscht werden müssen. Ein Ziel: Langfristig sollen Strecken und Schienenknoten mit dem europäischen Zugbeeinflussungssystem ETCS, das steht für „European Train Control System“, ausgerüstet werden. Zudem sollen digitale Stellwerke eingebaut werden.\n\nMehr Kapazität durch Modernisierung\n\nDas soll laut Website des DB-Programms helfen, ohne neue Gleise mehr Züge auf die Strecken zu bekommen. Positiv auswirken soll es sich außerdem auf die Zuverlässigkeit im Güter- und Personentransport. Konkret soll die Modernisierung für 35 Prozent mehr Kapazität sorgen.\n\nGelder werden auf verschiedenen Strecken eingesetzt\n\nDas Geld wird unter anderem auf dem Güterverkehrskorridor Rhein-Alpen eingesetzt – da soll das europäischen Eisenbahnverkehrsleitsystem ERTMS, kurz für „European Rail Traffic Management System“, eingesetzt werden. Gelder in Höhe von etwa 307 Millionen Euro sind für Modernisierungen des Korridors Skandinavien-Mittelmeer geplant.\n\nEtwa 83 Millionen Euro sollen für den „Digitalen Knoten Stuttgart“ genutzt werden. Elf Millionen Euro sind dann noch für die Schnellfahrstrecke Köln-Frankfurt/Main einkalkuliert.",
        tgt_text = "Geld für moderne Schienen: Deutsche Bahn und Verkehrsministerium erhöhen Milliardenfinanzierung"
    ),
    InputExample(
        guid = 3,
        text_a = "Dass ein Asteroideneinschlag auf der Erde verheerende Folgen haben könnte, dürfte allen klar sein. Vorgeführt zu bekommen, was genau ein solches Ereignis auslösen könnte, ist aber noch einmal beeindruckender. Genau dafür sorgt eine neu entwickelte Website.\n\nPunktgenaue Simulation\n\nDas Projekt nennt sich „Asteroid Launcher“ und wurde von Neal Agarwal entwickelt, der ein Händchen für faszinierende Datenprojekte hat.\n\nMan kann damit nicht nur punktgenau simulieren, wo der Asteroid einschlägt, sondern auch zahlreiche weitere Parameter eingeben: Aus welchem Material soll der Asteroid bestehen? Eisen? Stein? Oder etwa Gold? Welchen Durchmesser soll er haben, mit welcher Geschwindigkeit und in welchem Winkel soll er auf die Erde prallen?\n\nErdbeben und Tornados könnten auf Einschlag folgen\n\nDann berechnet die Seite, welche enorme Schäden er anrichten würde. Die Ergebnisse, die dabei präsentiert werden, sind erschütternd und grotesk, denn es werden nicht nur die Zahlen der vom Einschlag hervorgerufenen Todesopfer präsentiert, sondern auch die Folgeauswirkungen des Einschlages mit einkalkuliert.\n\nEin solcher könnte nämlich auch Erdbeben und Tornados zur Folge haben, ganz zu schweigen von der Schockwelle und dem Feuerball, den er auslösen würde.\n\nNasa könnte über das Projekt erfreut sein\n\nAgarwal hat sein neuestes Projekt auf Twitter lapidar mit den Worten „Bauen Sie ihren eigenen Asteroiden und lassen sie ihn auf die Erde rasen, um zu sehen, welche Auswirkungen er hat“ angekündigt.\n\nSo gruselig die Seite anmutet, die Nasa könnte darüber erfreut sein. Laut der Technologie-Plattform Futurism wird den Menschen dadurch vermittelt, welch verheerende Auswirkungen solch ein Szenario haben könnte. Und das könnte der Nasa in der öffentliche Wahrnehmung helfen, ihre große Investitionen, die sie zur Prävention solcher potenziellen Einschläge tätigt, zu legitimieren.",
        tgt_text = "Asteroideneinschlag in deiner Stadt: Hier erfährst du, was passieren würde"
    ),
    InputExample(
        guid = 4,
        text_a = "„Deutschland soll führender Startup-Standort in Europa werden“, so haben es die Ampel-Parteien im November 2021 in ihren Koalitionsvertrag für die kommenden vier Jahre geschrieben. Zudem sollte sich Deutschland laut SPD, Grünen und FDP in ein klimaneutrales Industrieland transformieren: „Die Klimaschutzziele von Paris zu erreichen, hat für uns oberste Priorität. Klimaschutz sichert Freiheit, Gerechtigkeit und nachhaltigen Wohlstand“, hieß es von der Ampel.\n\nWas ist seitdem passiert?\n\nEin Jahr Koalitionsvertrag – so steht es um die Versprechen für die Startup-Szene\n\nEin Jahr Koalitionsvertrag – wir haben auf Basis eines Papiers des Startup-Verbands, das t3n vorliegt, einige der wichtigsten Änderungen der letzten zwölf Monate zusammengefasst.\n\n1. Startup-Strategie\n\nNoch in den ersten 100 Tagen hat die Bundesregierung, die im Koalitionsvertrag angekündigte „umfassende Startup-Strategie“ in Angriff genommen. Im Juli hat das Bundeskabinett dann erstmals überhaupt eine Startup-Strategie einer Bundesregierung verabschiedet.\n\n„Wir bewerten die schnelle Verabschiedung der Startup-Strategie, in deren Erarbeitung wir als Startup-Verband intensiv eingebunden wurden, insgesamt positiv. Damit hat sich die Ampel eine eigene To-Do-List erstellt. Nun kommt es auf eine schnelle Umsetzung an“, kommentiert der Startup-Verband.\n\nIn der Strategie werden für zehn Themenfelder Handlungsvorhaben definiert, die innerhalb der Legislaturperiode umgesetzt werden. Dazu gehören unter anderem die Themen Startup-Finanzierung, Diversität und Mitarbeiterbeteiligung.\n\n2. ESOP/Mitarbeiterkapitalbeteiligungen\n\n„Wir werden die Mitarbeiterkapitalbeteiligung für Start-ups attraktiver gestalten“, heißt es unmissverständlich im Koalitionsvertrag. Bereits kurz nach Amtsantritt Ende 2021 bekräftigte der dafür federführende Bundesfinanzminister Christian Lindner das Versprechen und weckte damit Erwartungen auf eine schnelle Umsetzung. Doch bislang gibt es hierzu noch keinen Entschluss.\n\nIm Sommer wurden Eckpunkte zum sogenannten „Zukunftsfinanzierungsgesetz“ vorgestellt, die jedoch laut Startup-Verband wichtige Bereiche, wie zum Beispiel die Vermeidung der sogenannten Dry-Income-Besteuerung, außer Acht ließen.\n\nMit einem neuen Referent:innenentwurf wird noch in diesem Jahr gerechnet. Das Gesetzgebungsverfahren soll nach den aktuellen Planungen in der ersten Jahreshälfte 2023 abgeschlossen werden, damit das Gesetz zum Ende nächsten Jahres in Kraft treten kann.\n\nDie Verbesserung der Rahmenbedingungen von Mitarbeiterkapitalbeteiligungen seien für Deutschland von grundlegender Bedeutung, um die Wettbewerbsfähigkeit als Startup-Standort zu sichern, so der Startup-Verband.\n\n3. Universitäten als Startup-Schmieden\n\nLaut Koalitionsvertrag sollten Universitäten eine wichtigere Rolle übernehmen, wenn es darum geht, junge Menschen für Gründungen zu begeistern. Für einen „echten Innovationsschub“ sollten Ausgründungen vorangetrieben und Mittel des Bundes „zur Schaffung einer Gründungsinfrastruktur für technologisches wie soziales Unternehmertum“ bereitgestellt werden.\n\nDie Startup-Strategie knüpft daran an. Danach soll unter anderem „in Ergänzung zu EXIST-Potentialen“ ein Leuchtturmwettbewerb für Entrepreneurship-Center aufgelegt werden. Die Konzeptphase soll 2023 beginnen und abgeschlossen werden. Hierfür wurde bereits eine Kommission einberufen, die zeitnah Ergebnisse präsentieren soll.\n\n„Für den Erfolg des Projektes wird entscheidend sein, die dafür erforderlichen Haushaltsmittel ab 2024 zur Verfügung zu stellen“, schreibt der Startup-Verband in seiner Einschätzung vom 7. Dezember. „Neben der Förderung von Hochschul-Ausgründungen sehen wir auch beim Transfer von außeruniversitären Forschungsinstituten viel ungenutztes Potential, das dringend genutzt werden sollte.“\n\nDazu gehöre auch, den IP-Transfer zu vereinfachen. Um das zu erreichen, konnte mit dem Start der Initiative „IP for virtual shares“ von SprinD ein erster wichtiger Beitrag geleistet werden.\n\n4. Cannabislegalisierung\n\nVon vielen mit großer Begeisterung wurde das Kapitel des Koalitionsvertrags aufgenommen, in dem es um die Legalisierung von Cannabis ging. „Die kontrollierte Abgabe von Cannabis an Erwachsene zu Genusszwecken in lizenzierten Geschäften“ sollte laut Koalitionsvertrag möglich gemacht werden.\n\nUm das zu erreichen, hat Gesundheitsminister Karl Lauterbach Ende Oktober 2022 ein Eckpunktepapier vorgelegt. „Die Drogenpolitik muss erneuert werden. Wir wollen den Cannabis-Konsum unter Gesundheitsaspekten reformieren“, sagte Lauterbach dazu. Cannabis und THC sollen künftig nicht mehr als Betäubungsmittel (BtM) eingestuft werden.\n\nIm Jahr 2021 schätzte der Medizin-Professor und FDP-Politiker Andrew Ullmann die Steuereinnahmen durch Cannabis auf circa 1,3 Milliarden Euro pro Jahr.\n\nBis es zur Cannabislegalisierung in Deutschland kommt, gibt es allerdings noch einige Hürden zu überwinden.\n\n„Das größte Fragezeichen beim vorliegenden Eckpunktepapier ist und bleibt die Interpretation durch die Europäische Kommission“, sagt Cannabis-Unternehmer Niklas Kouparanis im Podcast mit t3n.\n\n5. Unternehmensgründung in 24 Stunden\n\n",
        tgt_text = "Ein Jahr Koalitionsvertrag – so steht es um die Versprechen für die Startup-Szene"
    ),
]

# Split: train 50%, test 50%
train_dataset = dataset[0:2]
test_dataset = dataset[2:4]

#### Option 2: Load dataset from drive

First mount drive to access dataset files stored in google drive.

In [7]:
# Mount drive to load datasets from drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Then load and adapt the dataset from drive. Adaptation is neccesary to transform data into  OpenPromts ```InputExample``` format. \
We used a dataset with 1000 datapoints and a 90/10 train/test split. 

*Note: If you try to reproduce this, you will need to store your own dataset in drive. \
Most likely yo will need to write your own ```adapt_dataset``` function too.*

In [8]:
# Load and adapt datasets
# split: train 90%, test 10%

def adapt_dataset(path_to_file):
  dataset = []
  with open(path_to_file, "r") as f:
    data = f.read()
    examples = data.split("[Text]:")
    examples = examples[1:]
    for guid, example in enumerate(examples):
      fulltext, title = example.split("[Titel]:")

      dataset.append(InputExample(
            guid = guid,
            text_a = fulltext,
            tgt_text = title.strip("\n\t\t\t\t\t").strip("\n\"")
        ))
  return dataset
  
train_dataset = adapt_dataset("/content/drive/MyDrive/train.csv")
test_dataset = adapt_dataset("/content/drive/MyDrive/test.csv")

##### Optional: Have a look at the data

In [10]:
# Have a look at the data (optional)
print(f"Train Examples: {train_dataset[:3]}")
print(f"Data points in train dataset: {len(train_dataset)}")
print(f"Test Examples: {test_dataset[:3]}")
print(f"Data points in test dataset: {len(test_dataset)}")

Train Examples: [{
  "guid": 0,
  "label": null,
  "meta": {},
  "text_a": "Berlin. Die Klagen der Fans waren laut und zahlreich. Zu soft, zu beliebig sei die neue Single, hie\u00df es in den Sozialen Netzwerken. F\u00fcr eine Band wie Linkin Park nichts Neues.\nOft schon haben die Amerikaner mit Stilen gespielt und Erwartungen entt\u00e4uscht. Wohin steuert die Band mit ihrem neuen Album \"\"One More Light\"\" (Warner)?\nHarte Gitarrenriffs, schn\u00f6rkelloser Rock? All das sucht man auf dem siebten Studioalbum vergeblich. Mit dem Vorg\u00e4nger \"\"The Hunting Party\"\" waren Linkin Park noch zu ihren Wurzeln zur\u00fcckgekehrt; \"\"Heavy\"\", die erste Single, hingegen ist astreiner Radiopop. Chester Bennington singt schwerm\u00fctig \u00fcber seine Gedanken. Im hymnischen Refrain fragt er, warum alles so schwer sei. Dann \u00fcbernimmt die 21-j\u00e4hrige S\u00e4ngerin Kiara als weiblicher Konterpart. Schema F.\nDie Reaktionen der Fans? \"\"Was ein Witz, Linkin Park. Hoffentlich k

###Step 2: Obtain a Pretrained Language Model (PML). 

The GPT-J model and tokenizer are not in the OpenPrompt default configuration. So we load them separately adapting from this [Tutorial](https://github.com/thunlp/OpenPrompt/blob/main/tutorial/6.1_chinese_dataset_uer_t5.py).

In [9]:
# Here the combination of tokenizer and model is not in the default configurations of openprompt.
# So we load the tokenizer and model separately.
from transformers import AutoTokenizer
from openprompt.plms.lm import LMTokenizerWrapper

# We use the model configuration and the tokenizer of GPT-J-6B and set pad_token as eos_token.
config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6B")
config.pad_token_id = config.eos_token_id
# Parameter setting use_fast=False is neccessary for usage with OpenPromt Templates later
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B", add_special_tokens=False, use_fast=False)
tokenizer.pad_token = config.pad_token_id

# We load the pre-trained gpt-j-6b with 8-bit weights from huggingface. 
# To reduce the peak RAM usage, we add the argument, low_cpu_mem_usage=True to from_pretrained.
model = GPTJForCausalLM.from_pretrained("hivemind/gpt-j-6B-8bit", low_cpu_mem_usage=True)

# Set tokenizer wrapper
tokenizer_wrapper = LMTokenizerWrapper(max_seq_length=2048, tokenizer=tokenizer)

k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, 

##### Optional: Quick test to check that model generation works

In [ ]:
# Just a quick test, to check whether generation works (optional)
from transformers import Text2TextGenerationPipeline

# Check if gpu ia available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# Move model accordingly
model = model.to(device)

# Initialize text generation pipeline and generate a test text
text2text_generator = Text2TextGenerationPipeline(model, tokenizer, device=0)
generated_text = text2text_generator("Hallo, mein Name ist", max_length=50, do_sample=False)
print(generated_text)

### Step 3: Define a Template

A Template defines how to modify the original text for input. It is one of the most important modules in prompt-learning.

In [10]:
from openprompt.prompts import ManualTemplate, PtuningTemplate

promptTemplate = ManualTemplate(
    text = '{"soft": None} {"soft": "Titel"} {"soft": None} {"soft": "folgendem"} {"soft": "Zeitungsartikel:"} {"placeholder":"text_a"} {"mask"}',
    tokenizer = tokenizer,
)

### Step 4: Construct a Prompt Model

Given the task, now we have a PLM and a Template, so we combine them into a ```PromptModel```.

In [11]:
use_cuda = True
prompt_model = PromptForGeneration(plm=model,template=promptTemplate, freeze_plm=False, plm_eval_mode=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()

### Step 5: Define a DataLoader

A ```PromptDataLoader``` is basically OpenPrompts version of pytorch Dataloader, which also includes a Tokenizer and a Template.

In [12]:
# Define DataLoader

from openprompt import PromptDataLoader

train_dataloader = PromptDataLoader(
    dataset = train_dataset,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=LMTokenizerWrapper,
)
validation_dataloader = PromptDataLoader(
    dataset = test_dataset,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=LMTokenizerWrapper,
)

tokenizing: 0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (5531 > 2048). Running this sequence through the model will result in indexing errors
tokenizing: 990it [00:06, 156.79it/s]
tokenizing: 10it [00:00, 135.92it/s]


##### Optional: Visualize some encoded data

In [13]:
# visualize some encoded data (optional)
print(next(iter(train_dataloader)))
print(tokenizer.decode(next(iter(train_dataloader))['input_ids'][0]))
#print(tokenizer.decode(next(iter(train_dataloader))['decoder_input_ids'][0]))

{"input_ids": [[11307, 13, 6733, 14770, 11286, 4587, 24964, 266, 5757, 300, 2306, 3318, 1976, 15668, 260, 488, 13, 1168, 84, 2705, 11, 1976, 84, 1250, 14261, 384, 72, 4656, 497, 518, 14206, 11, 289, 494, 39683, 1658, 287, 2853, 1406, 89, 498, 268, 3433, 89, 15448, 3464, 13, 376, 25151, 304, 500, 10243, 266, 494, 7502, 259, 3250, 299, 488, 912, 3169, 947, 13, 198, 46, 701, 5513, 261, 387, 11722, 4656, 33391, 49894, 263, 10255, 520, 346, 268, 308, 9774, 72, 2120, 3318, 5256, 24657, 2150, 268, 920, 83, 11033, 385, 21474, 13, 370, 1219, 259, 2876, 84, 861, 4656, 10243, 10255, 1312, 71, 2787, 497, 84, 268, 23999, 13538, 3198, 3125, 4401, 15931, 357, 13195, 1008, 19427, 198, 39, 32074, 402, 7940, 918, 81, 10203, 11, 264, 1349, 30570, 17164, 13416, 4631, 30, 1439, 288, 292, 884, 83, 582, 257, 3046, 1357, 264, 494, 65, 1452, 11733, 40916, 27373, 2436, 488, 13, 11707, 1357, 569, 2398, 11033, 782, 263, 13538, 464, 22059, 3615, 15931, 266, 5757, 7502, 259, 3250, 645, 354, 1976, 84, 1312, 71, 918,

We are done with the setup!

## Training Loop for Prompt Tuning

We prompt tune the model and save it 👏

Define a filepathe for saving the model.

In [14]:
# Define a filepath for saving the model
filepath = '/content/drive/MyDrive/soft-prompt-tuned-model-1000-2.pt'

Run the training loop.

In [15]:
print("truncate rate: {}".format(train_dataloader.tokenizer_wrapper.truncate_rate), flush=True)


generation_arguments = {
    "max_new_tokens": 30,
}

def evaluate(prompt_model, dataloader):
    predictions = []
    ground_truths = []

    for step, inputs in enumerate(dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        _, output_sentence = prompt_model.generate(inputs, **generation_arguments, verbose=False)
        predictions.extend(output_sentence)
        ground_truths.extend(inputs['tgt_text'])
    assert len(predictions)==len(ground_truths), (len(predictions), len(ground_truths))
    predictions = [prediction.strip() for prediction in predictions]
    ground_truths = [ground_truth.strip() for ground_truth in ground_truths]
    # shown one example
    print(f"predictions {predictions[0]}, ground_truths {ground_truths[0]}")
    score = sum([prediction[:len(ground_truth)]==ground_truth for prediction, ground_truth in zip(predictions, ground_truths)])/len(ground_truths)
    return score


from transformers import get_linear_schedule_with_warmup,get_constant_schedule_with_warmup  # use AdamW is a standard practice for transformer
from transformers.optimization import Adafactor  # use Adafactor is the default setting for T5

# This function never gets called?!
loss_func = torch.nn.CrossEntropyLoss()

# Define Training Parameters; initital definittion from tutorial
seed = 144
result_file="../results.txt"
max_steps=5000 #50
lr=1e-2
warmup_step_prompt=500 #5
eval_every_steps=500 #5
tot_step = max_steps
num_epochs = 1000000

# normally we freeze the model when using soft_template. However, we keep the option to tune plm
no_decay = ['bias', 'LayerNorm.weight'] # it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters1 = [
    {'params': [p for n, p in prompt_model.plm.named_parameters() if (not any(nd in n for nd in no_decay))], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.plm.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# we should use optimizer_grouped_parameters1, for the Adam Optimizer 
#optimizer1 = AdamW(optimizer_grouped_parameters1, lr=1e-2)
#optimizer1 = Adam8bit(model.parameters(), lr=lr, weight_decay=0.01)
optimizer1 = Adam8bit(optimizer_grouped_parameters1, lr=lr, weight_decay=0.01)
scheduler1 = get_linear_schedule_with_warmup(
    optimizer1,
    num_warmup_steps=500, num_training_steps=tot_step)

tot_loss = 0
log_loss = 0
best_val_acc = 0
glb_step = 0
actual_step = 0
leave_training = False
gradient_accumulation_steps = 1

acc_traces = []
tot_train_time = 0
pbar_update_freq = 50
prompt_model.train()

pbar = tqdm(total=tot_step, desc="Train")
for epoch in range(num_epochs):
    for step, inputs in enumerate(train_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        tot_train_time -= time.time()
        loss = prompt_model(inputs)
        loss.backward()
        tot_loss += loss.item()
        actual_step += 1

        if actual_step % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(prompt_model.parameters(), 1.0)
            glb_step += 1
            if glb_step % pbar_update_freq == 0:
                aveloss = (tot_loss - log_loss)/pbar_update_freq
                pbar.update(pbar_update_freq)
                pbar.set_postfix({'loss': aveloss, "epoch": epoch})
                log_loss = tot_loss


                if optimizer1 is not None:
                    optimizer1.step()
                    optimizer1.zero_grad()
                if scheduler1 is not None:
                    scheduler1.step()

        tot_train_time += time.time()

        if actual_step % gradient_accumulation_steps == 0 and glb_step >0 and glb_step % eval_every_steps == 0:
            val_acc = evaluate(prompt_model, validation_dataloader)
            if val_acc >= best_val_acc:
                # Uncomment lines below to save model checkpoints
                print('save')
                torch.save(prompt_model, filepath)
                best_val_acc = val_acc

            acc_traces.append(val_acc)
            print("Glb_step {}, val_acc {}, average time {}".format(glb_step, val_acc, tot_train_time/actual_step ), flush=True)
            prompt_model.train()

        if glb_step > max_steps:
            leave_training = True
            break

    if leave_training:
        break

truncate rate: 0.4858585858585859


Train:  10%|█         | 500/5000 [07:34<1:07:50,  1.11it/s, loss=0.463, epoch=0]

predictions ung der Verkehrswege einsparen. Das ist ein guter Anfang. Die Kosten für die, ground_truths Straubhaar: „Weniger Sozialstaat, dafür bessere Straßen“
save
Glb_step 500, val_acc 0.0, average time 0.9092238235473633


Train:  20%|██        | 1000/5000 [17:10<1:02:14,  1.07it/s, loss=0.307, epoch=1]

predictions ung der Verkehrswege einsetzen. Das ist eine gute Initiative, aber sie ist nicht a, ground_truths Straubhaar: „Weniger Sozialstaat, dafür bessere Straßen“
save
Glb_step 1000, val_acc 0.0, average time 0.9065740208625793


...

predictions ung

English

D

German

05.02.2019

In

De


-

+++, ground_truths Straubhaar: „Weniger Sozialstaat, dafür bessere Straßen“
save
Glb_step 4000, val_acc 0.0, average time 0.9044017723798752


Train:  90%|█████████ | 4500/5000 [1:15:19<07:37,  1.09it/s, loss=6.37e-7, epoch=4]

predictions ung

English

D

German

05.02.2019


Sch



Inf


-, ground_truths Straubhaar: „Weniger Sozialstaat, dafür bessere Straßen“
save
Glb_step 4500, val_acc 0.0, average time 0.9043386662801107


Train: 100%|██████████| 5000/5000 [1:23:37<00:00,  1.09it/s, loss=4.24e-7, epoch=5]

predictions ung

English

D

German

05.02.2019


Sch



Inf


-, ground_truths Straubhaar: „Weniger Sozialstaat, dafür bessere Straßen“
save
Glb_step 5000, val_acc 0.0, average time 0.9043051354408265


## Evaluation

To evaluate the prompt tuned model generate some headlines.

In [16]:
model.eval()

for step, inputs in enumerate(validation_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    _, output_sentence = prompt_model.generate(inputs, **generation_arguments, verbose=False)
    print(output_sentence)

['ung\n\nEnglish\n\nD\n\nGerman\n\n05.02.2019\n\n\nSch\n\n\n\nInf\n\n\n-']
['English\n\n-\n\n00:00:00\n\n-\n\n00:00:00\n\n-\n\nJ']
['English\n\nZ\n\nDe\n\n00:00\n\nSR\n\nSW\n\n00\n\nM\n\nM']
['English\n\nM\n\nA\n\nZ\n\nCategory\n\n(FR\n\nSP\n\n)\n\nJuly']
['04.02.84']
['ende\n\nN']
['English\n\nBerlin\n\n07.02.2019\n\nA\n\n\nns']
['']
['']
['N\n\n###\n\n###\n\n###\n\n###\n\n###\n\n###\n\n###\n\n###\n\n###']
